# Chat bot question answering.

The goal of your project is to create a chatbot model that provides answers on client questions.   
Your goal is to divide your dataset on several sections and to make multi-level classifier that will classify the section and later the most reasonable(closest) answer.    
Take care about text-preprocessing, stop words removal, and sentence vectorization

This project consists of the following activities :  




1. Phase 1 : Dataset
    * Team Planning
    * Full git project Integration
    * General Project Research
    * Dataset Collection
    * Dataset Preparation
2. Phase 2 : Training
    * Research about NLP model
    * Compose NLP model
        * Stop words Removal
        * Text tokenization
        * Text Preprocessing
        * Question vectorization
        * Find closest vector
    * Ping Pong phase with Dataset labelers
    * Generate more data if needed
    * Fine tunning of your model


3. Phase 3 : Deployment
    * Perform manual benchmark
    * Model Deploy (Git)
    * Write git Readme.md file
    * Receive Feedback from PM


In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy.linalg as la
import re

%matplotlib inline

# Load Data

In [2]:
brainster_df = pd.read_csv('dataset/dataset_brainster.csv')

In [3]:
brainster_df

,questions,answers,category,category_id
0,Колку време трае академијата за дигитален марк...,Академијата за дигитален маркетинг трае 23 нед...,маркетинг,1
1,колку месеци недели е академијата за дигитален...,Академијата за дигитален маркетинг трае 23 нед...,маркетинг,1
2,Како се одвива наставата на академијата за диг...,Наставата за академијата за дигитален маркетин...,маркетинг,1
3,Дали наставата за маркетинг е online,Наставата за академијата за дигитален маркетин...,маркетинг,1
4,Колку часа неделно има на маркетинг академијата,Неделно се одржуваат три часови во времетрање ...,маркетинг,1
...,...,...,...,...
518,Дали добивам диплома по завршување на Академија?,"Ако успешно го одбраниш завршниот проект, доби...",UX/UI,6
519,Како да аплицирам на академијата за UX/UI?,"Со пополнување на формуларот за апликација, вл...",UX/UI,6
520,Како ги одбирате студентите на академијата за ...,По средбата или интервјуто координаторот има с...,UX/UI,6
521,Дали може да се приклучам на листа на кандидат...,Да. Отворени сме за апликации и кога е групата...,UX/UI,6


# Provide and prepare data information

In [4]:
questions = brainster_df.questions
other_col = brainster_df.drop(columns='questions', axis=0)

In [5]:
questions.shape

(523,)

In [6]:
other_col.shape

(523, 3)

In [7]:
other_col.head(5)

,answers,category,category_id
0,Академијата за дигитален маркетинг трае 23 нед...,маркетинг,1
1,Академијата за дигитален маркетинг трае 23 нед...,маркетинг,1
2,Наставата за академијата за дигитален маркетин...,маркетинг,1
3,Наставата за академијата за дигитален маркетин...,маркетинг,1
4,Неделно се одржуваат три часови во времетрање ...,маркетинг,1


In [8]:
questions.head(5)

0    Колку време трае академијата за дигитален марк...
1    колку месеци недели е академијата за дигитален...
2    Како се одвива наставата на академијата за диг...
3                 Дали наставата за маркетинг е online
4      Колку часа неделно има на маркетинг академијата
Name: questions, dtype: object

In [9]:
questions_array = questions.to_numpy()
questions_array

array(['Колку време трае академијата за дигитален маркетинг?',
       'колку месеци недели е академијата за дигитален маркетинг',
       'Како се одвива наставата на академијата за дигитален маркетинг?',
       'Дали наставата за маркетинг е online',
       'Колку часа неделно има на маркетинг академијата',
       'Каква опрема ми е потребна за академијата за дигитален маркетинг?',
       'што ми треба за да слушам дигитален маркетинг',
       'Што се изучува на академијата за дигитален маркетинг?',
       'што се учи на дигитален маркетинг',
       'Дали има слободни места на академијата за дигитален маркетинг?',
       'дали има место да се запишам на дигитален маркетинг?',
       'Кога почнуваат уписите за следната академија за дигитален маркетинг?',
       'Дали постои можност за вработување после завршувањето на Академијата?',
       'дали посредувате при вработување на маркетинг',
       'Дали наставата вклучува само работа на компјутер или има и некои други активности?',
       

# Data Preprocessing

In [191]:
lat_to_cyr = {'kj' : 'ќ', 'gj' : 'ѓ', 'zh' : 'ж', 'ch' : 'ч', 'sh' : 'ш', 'dj' : 'ѓ',
              'a' : 'а', 'b' : 'б', 'c' : 'ц', 'd' : 'д', 'e' : 'е', 'f' : 'ф', 'g' : 'г',
              'h' : 'х', 'i' : 'и', 'j' : 'ј', 'k' : 'к', 'l' : 'л', 'm' : 'м', 'n' : 'н',
              'o' : 'о', 'p' : 'п', 'q' : 'љ', 'r' : 'р', 's' : 'с', 't' : 'т', 'u' : 'у',
              'v' : 'в', 'w' : 'њ', 'x' : 'џ', 'y' : 'ѕ', 'z' : 'з'
             }


def latin_to_cyriluc(text):
    questions = []
    for question in text:
        for key, value in lat_to_cyr.items():
            question = re.sub(key, value, question.lower())
        questions.append(question)
    return questions

In [192]:
latin_to_cyriluc(questions)

['колку време трае академијата за дигитален маркетинг?',
 'колку месеци недели е академијата за дигитален маркетинг',
 'како се одвива наставата на академијата за дигитален маркетинг?',
 'дали наставата за маркетинг е онлине',
 'колку часа неделно има на маркетинг академијата',
 'каква опрема ми е потребна за академијата за дигитален маркетинг?',
 'што ми треба за да слушам дигитален маркетинг',
 'што се изучува на академијата за дигитален маркетинг?',
 'што се учи на дигитален маркетинг',
 'дали има слободни места на академијата за дигитален маркетинг?',
 'дали има место да се запишам на дигитален маркетинг?',
 'кога почнуваат уписите за следната академија за дигитален маркетинг?',
 'дали постои можност за вработување после завршувањето на академијата?',
 'дали посредувате при вработување на маркетинг',
 'дали наставата вклучува само работа на компјутер или има и некои други активности?',
 'дали е потребно предзнаење за успех на академијата за дигитален маркетинг?',
 'кои се инструкто

In [144]:
symbols = (u"абвгдѓежзѕијклљмнњопрстќуфхцчџш",
           u"abvgd]e\zyijklqmnwoprst'ufhc;x[")

tr = {ord(a):ord(b) for a, b in zip(*symbols)}

tr

{1072: 97,
 1073: 98,
 1074: 118,
 1075: 103,
 1076: 100,
 1107: 93,
 1077: 101,
 1078: 92,
 1079: 122,
 1109: 121,
 1080: 105,
 1112: 106,
 1082: 107,
 1083: 108,
 1113: 113,
 1084: 109,
 1085: 110,
 1114: 119,
 1086: 111,
 1087: 112,
 1088: 114,
 1089: 115,
 1090: 116,
 1116: 39,
 1091: 117,
 1092: 102,
 1093: 104,
 1094: 99,
 1095: 59,
 1119: 120,
 1096: 91}

In [141]:
text = 'kaj si'
print(text.lower().translate(lat_to_cyr))

kaj si


In [10]:
stop_words_mkd = pd.read_csv('stop_words.txt').to_numpy()
stop_words_list = []
for words in stop_words_mkd:
    for word in words:
        stop_words_list.append(word)
        
stop_words_list

['еден',
 'едно',
 'јас',
 'рече',
 'сите',
 'сум',
 'ние',
 'сме',
 'овде',
 'исто',
 'така',
 'на',
 'и',
 'во',
 'се',
 'го',
 'не',
 'да',
 'е',
 'што',
 'од',
 'со',
 'за',
 'или',
 'тоа',
 'ја',
 'по',
 'тој',
 'но',
 'а',
 'ги',
 'до',
 'таа',
 'му',
 'кои',
 'па',
 'нив',
 'ни',
 'ќе',
 'кој',
 'итн',
 'бил',
 'кај',
 'ова',
 'врз',
 'над',
 'тие',
 'a',
 'две',
 'би',
 'она',
 'неа',
 'си',
 'кое',
 'ако',
 'два',
 'има',
 'в',
 'како',
 'само',
 'дека',
 'една',
 'туку',
 'кога',
 'сега',
 'ми',
 'потоа',
 'низ',
 'ти',
 'кон',
 'додека',
 'веќе',
 'нешто',
 'него',
 'уште',
 'таму',
 'ли',
 'под',
 'беа',
 'ме',
 'некој',
 'ништо',
 'тука',
 'пред',
 'им',
 'каде',
 'повторно',
 'ниту',
 'биде',
 'толку',
 'никогаш',
 'мене',
 'дали',
 'тогаш',
 'своите',
 'сето',
 'без',
 'нема',
 'околу',
 'многу',
 'полека',
 'секогаш',
 'зошто',
 'те',
 'добро',
 'можеби',
 'колку',
 'можеше',
 'нивните',
 'преку',
 'миг',
 'знам',
 'малку',
 'вратата',
 'ох',
 'навистина',
 'оваа',
 'по

# Define Model Architecture

Count Vectorizer Model

In [11]:
count_vector = CountVectorizer(stop_words=stop_words_list, strip_accents='unicode')

In [12]:
count_vector

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['еден', 'едно', 'јас', 'рече', 'сите', 'сум', 'ние',
                            'сме', 'овде', 'исто', 'така', 'на', 'и', 'во',
                            'се', 'го', 'не', 'да', 'е', 'што', 'од', 'со',
                            'за', 'или', 'тоа', 'ја', 'по', 'тој', 'но', 'а', ...],
                strip_accents='unicode', token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [63]:
questions_features = count_vector.fit_transform(questions_array)
questions_df = pd.DataFrame(data = questions_features.todense(), 
                            columns=count_vector.get_feature_names())

questions_df.tail(10)

,15,16,24,30,ads,ajax,appium,automated,back,basics,...,цената,чао,часа,часови,часовите,чини,школувањето,јазик,јазици,јак
513,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
514,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
517,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
questions_df.shape

(523, 509)

In [15]:
# display function
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [16]:
display_all(questions_df.head(2))

,15,16,24,30,ads,ajax,appium,automated,back,basics,bi,big,bootstrap,brainster,content,copywriting,css,cv,data,deep,digital,end,facebook,front,full,fundamentals,generation,git,google,growth,hacking,html,instagram,it,java,jquery,laravel,layout,lead,learning,linkedin,machine,marketing,maven,of,online,partner,php,popup,python,qa,react,real,science,selenium,seo,software,sql,stack,storytelling,strategy,tester,testing,testng,time,ui,ux,webdriver,website,автоматско,академии,академиите,академија,академијата,акредитиран,акредитирана,активности,анализа,ангажирани,англиски,аплицира,аплицираат,аплицирам,бази,банкарски,бидам,бидат,број,важи,ваучер,ваучерот,вашата,ваши,ве,вебсајт,веке,видам,видат,видео,вирусот,вклучува,владата,внимание,вработување,време,времетрање,врши,гледам,години,готово,графички,група,групата,групи,давате,датабази,датум,делот,деновиве,детали,детална,дигитален,дигиталниот,дизајн,дизајни,диплома,дипломата,добар,добива,добивам,добиен,добијам,дознаам,доколку,доколу,достапна,другите,економски,живеам,живо,завршување,завршувањето,заинтересиран,заинтересирана,замолам,запишам,запишеме,запишување,започнам,започнеме,започнува,здраво,земјава,знаел,знаења,знаење,избираат,избирате,изработам,изработка,изработуваат,изучува,изучуваат,изучувате,имаат,имам,имаме,имате,инсталирање,инструктори,инструкторите,интересира,инфо,информации,информација,искористи,искусни,искуство,исполнувам,кажете,каква,какви,какво,каков,камера,кандидат,кандидати,кандидатите,кариерен,картичка,ке,клиенти,книги,кого,колкави,колки,компании,компјутер,компјутери,конкретно,контакт,контактирам,концепирана,користам,користат,користи,корона,кошта,кратки,кратко,крај,крајот,кредитна,креирање,купам,купи,курс,курсеви,курсевите,лап,лаптоп,лето,листа,лица,лого,маил,македонија,македонски,маркетинг,математика,машински,мегународно,месеци,места,место,мк,млади,можам,можат,можност,можностите,моите,мокен,молам,моменталната,мрежи,надвор,надоградам,наменет,наменета,наогате,направи,напреден,напредна,напредни,наставата,начин,наши,нашите,најдобро,најмногу,недели,неделно,некого,некои,немам,неопходно,нов,нуди,нудите,обврски,областа,области,обработуваат,обукaтa,обука,обуката,објектно,одбиен,одбираат,одбирате,одвива,одвиваат,одговор,одлука,одлучам,одлучи,однос,односно,одржува,одржуваат,онлајн,ооп,опрема,опција,организирате,ориентирано,официјалните,оценка,оценува,оценување,пазарот,пакување,пари,пат,плака,плакам,плакање,плакањето,платам,побарам,повеке,податоци,подготвам,подготвителна,подготвителната,подготовка,поддршка,подобра,подразбира,подразбора,поздрав,познавања,познавање,помегу,помогне,помогнете,помош,пополнета,попуст,поради,поразговарм,посветува,поседувам,посета,посредувате,посредување,постер,постои,поточно,потребен,потребна,потребни,потребно,почетник,почетници,почнала,почнам,почнува,почнуваат,појаснување,прават,правата,правен,правење,правите,пракса,практични,практично,прати,прашам,прашувам,прегледам,предава,предавачи,предавања,предавањата,предавање,предвидена,предзнаења,предзнаење,предложиме,предност,предноста,прекине,претставува,приватни,придобивките,признаваат,признат,призната,приклучам,примите,причина,пријавам,пријават,пријави,пријавување,прогамер,програм,програма,програмата,програмер,програмирам,програмирање,програмирањето,програмирње,програмски,прогресот,проект,проекти,профил,работа,работам,работат,работата,работи,работни,работно,разлика,рати,регистрацијата,резиме,рок,саат,саатот,сакал,сакала,сакам,самостојно,свој,сектор,селектирате,селекција,сертификат,сертификатот,ситуација,ситуацијата,скајп,скопје,следам,следат,следење,следи,следната,следни,следниот,следува,слично,слободни,случува,слушам,сопствен,соработка,соработува,соработувате,состојба,софтвер,социологија,социјални,специфични,спремате,средно,средношколец,средношколци,статистика,стекнува,стипендираат,стипендија,странство,студенти,студентите,студиите,телефон,теми,термин,тестер,тестирање,техничко,технологии,технологиите,тип,типографија,топ,траат,трае,траење,уписи,уписите,уплата,уплати,успех,учат,ученик,учесници,учесниците,учество,учест

# Test Part

In [130]:
test = ["каква кариера ме очекува по завршувањето на дата сајанс академијата"]
test_transform = count_vector.transform(test)
test_transform.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Define distance between two vectors

In [131]:
cosine_function = lambda a, b : round(np.inner(a, b)/(la.norm(a)*la.norm(b)), 3)

In [132]:
distances = []
for vector in questions_df.to_numpy():
    for testV in test_transform.toarray():
        cosine = cosine_function(vector, testV)
        distances.append(cosine)

# Example :  
   Find the closest dataset question based on user defined question

In [133]:
test

['каква кариера ме очекува по завршувањето на дата сајанс академијата']

In [134]:
distance = distances.index(max(distances))

In [135]:
other_col.answers[distance]

'Како дел од активностите на Академијата е и соработка со компании кои се наши партнети кои често нудат и можност за вработување.'

# Make Benchmark and provide info

# Make Summary about your results